In [1]:
hkmaisin=[
{
    "expected_maturity_date" : "2034-03-07",
    "original_maturity" : "15-year (15 年)",
    "isin_code" : "HK0000486388",
    "coupon" : "2.02"
},{
    "expected_maturity_date" : "2032-03-02",
    "original_maturity" : "15-year (15 年)",
    "isin_code" : "HK0000329679",
    "coupon" : "1.89"
},{
    "expected_maturity_date" : "2030-07-16",
    "original_maturity" : "15-year (15 年)",
    "isin_code" : "HK0000257821",
    "coupon" : "2.13"
},{
    "expected_maturity_date" : "2029-01-17",
    "original_maturity" : "10-year (10 年)",
    "isin_code" : "HK0000474293",
    "coupon" : "1.97"
},{
    "expected_maturity_date" : "2027-06-29",
    "original_maturity" : "10-year (10 年)",
    "isin_code" : "HK0000345436",
    "coupon" : "1.25"
},{
    "expected_maturity_date" : "2026-01-21",
    "original_maturity" : "10-year (10 年)",
    "isin_code" : "HK0000280708",
    "coupon" : "1.68"
},{
    "expected_maturity_date" : "2024-08-07",
    "original_maturity" : "10-year (10 年)",
    "isin_code" : "HK0000209236",
    "coupon" : "2.22"
},{
    "expected_maturity_date" : "2023-01-17",
    "original_maturity" : "10-year (10 年)",
    "isin_code" : "HK0000135431",
    "coupon" : "1.10"
},{
    "expected_maturity_date" : "2022-07-29",
    "original_maturity" : "3-year (3 年)",
    "isin_code" : "HK0000515046",
    "coupon" : "Floating (浮息)"
},{
    "expected_maturity_date" : "2022-05-18",
    "original_maturity" : "5-year (5 年)",
    "isin_code" : "HK0000339298",
    "coupon" : "1.16"
},{
    "expected_maturity_date" : "2021-12-17",
    "original_maturity" : "3-year (3 年)",
    "isin_code" : "HK0000464617",
    "coupon" : "Floating (浮息)"
},{
    "expected_maturity_date" : "2021-08-04",
    "original_maturity" : "10-year (10 年)",
    "isin_code" : "HK0000085537",
    "coupon" : "2.46"
},{
    "expected_maturity_date" : "2020-11-05",
    "original_maturity" : "5-year (5 年)",
    "isin_code" : "HK0000271582",
    "coupon" : "0.91"
},{
    "expected_maturity_date" : "2020-06-23",
    "original_maturity" : "3-year (3 年)",
    "isin_code" : "HK0000340619",
    "coupon" : "Floating (浮息)"
},{
    "expected_maturity_date" : "2020-02-05",
    "original_maturity" : "5-year (5 年)",
    "isin_code" : "HK0000235769",
    "coupon" : "1.06"
},{
    "expected_maturity_date" : "2020-01-13",
    "original_maturity" : "10-year (10 年)",
    "isin_code" : "HK0000059292",
    "coupon" : "2.93"
}]

In [2]:
import json

In [3]:
def transformYear(b):
    b['original_maturity'] = int(b['original_maturity'].split("-")[0])
    return b
hkmaisin_=list(map(lambda b: transformYear(b),hkmaisin))

In [4]:
from collections import defaultdict
TenorVsBond = defaultdict(list)
for b in hkmaisin_:
    TenorVsBond[ b['original_maturity'] ].append(b)

In [33]:
import random
import numpy as np
import uuid
import re
from typing import Dict
import time

class PrimaryInvestorRobot:
    def __init__ (self,name:str,TenorVsBond:Dict[int, Dict], tenorWeight:Dict[int, float], notional:int, std:float, bidask="ASK", ccy="HKD"):
        self.name = name
        self.TenorVsBond = TenorVsBond
        self.tenorWeight = tenorWeight
        self.notional = notional
        self.bidask = bidask
        self.std = std
        self.ccy = "HKD"
        self.YMDMatcher = re.compile(r"(\d{4})-(\d{2})-(\d{2})")
    def generateTrade(self):
        tenor = random.choices(list(tenorWeight.keys()), weights=list(tenorWeight.values()), k=1)[0]
        tradeid = str(uuid.uuid4())
        trade={}
        asset={}

        trade["id"] = tradeid
        trade["tradeType"] = "BOND"
        trade["timestamp"] = int(time.time() * 1000)
        security = random.choice(list(self.TenorVsBond[tenor]))
        
        mDate = self.YMDMatcher.match(security['expected_maturity_date'])
        if mDate is not None:
            year = int(mDate.group(1)) - tenor
            trade["tradeDate"] = f"{year}-{mDate.group(2)}-{mDate.group(3)}"
        
        asset['securityId'] = security['isin_code']
        asset['currency'] = self.ccy
        asset['price'] = float(security["coupon"])
        asset["bidask"] = self.bidask
        asset["notional"] = int(np.round(np.abs(np.random.normal(0, self.std, 1)), -3)[0])+self.notional
        trade['asset'] = asset
        
        return trade

In [34]:
tenorWeight = {15:0.3, 10:0.5, 5:0.2}
primaryInvestorRobot = PrimaryInvestorRobot("abcd",TenorVsBond,tenorWeight, 10000000,10000000)
trade = primaryInvestorRobot.generateTrade()

In [35]:
trade

{'id': 'c5253e6d-1c39-4ba2-9e0c-ffc165e22e07',
 'tradeType': 'BOND',
 'timestamp': 1580652874823,
 'tradeDate': '2016-01-21',
 'asset': {'securityId': 'HK0000280708',
  'currency': 'HKD',
  'price': 1.68,
  'notional': 18572000}}

## write to avro file
>Reference : http://layer0.authentise.com/getting-started-with-avro-and-python-3.html

In [36]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [37]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [38]:
schema = avro.schema.Parse(open("bondtrade.avsc", "rb").read())

In [39]:
writer = DataFileWriter(open("bondtrade.avro", "wb"), DatumWriter(), schema)
writer.append(trade);
writer.close()